In [1]:
import os

os.getcwd()
os.chdir("../")
os.getcwd()

'/home/mobo/Documents/Ineuron/Live_Class/DL/DL_live_notes/CNN/deepCNNClassifier'

In [2]:


from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_learning_rate: float

In [3]:
from deepClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from deepClassifier.utils import read_yaml, create_directories

class ConfigurationManager:
    
    def __init__(
        self, 
        config_filepath: Path=CONFIG_FILE_PATH, 
        params_filepath: Path=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_prepare_base_model(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        return PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_learning_rate=self.params.LEARNING_RATE
        )
    
    

In [4]:
config = ConfigurationManager()
print(config.config, config.params, "\n\n")
config.get_prepare_base_model()

[2022-09-21 10:12:32,067: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-09-21 10:12:32,071: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-21 10:12:32,073: INFO: common]: created directory at: artifacts
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzipped_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}} {'IMAGE_SIZE': [224, 224, 3], 'INCLUDE_TOP': False, 'WEIGHTS': 'imagenet', 'CLASSES': 2, 'LEARNING_RATE': 0.01} 


[2022-09-21 10:12:32,075: INFO: common]: created directory at: artifacts/prepare_base_model


PrepareBaseModelConfig(root_dir=PosixPath('artifacts/prepare_base_model'), base_model_path=PosixPath('artifacts/prepare_base_model/base_model.h5'), updated_base_model_path=PosixPath('artifacts/prepare_base_model/base_model_updated.h5'), params_image_size=BoxList([224, 224, 3]), params_include_top=False, params_weights='imagenet', params_classes=2, params_learning_rate=0.01)

In [6]:
from deepClassifier.constants import PARAMS_FILE_PATH
import tensorflow as tf

class PrepareBaseModel:
    def __init__(
        self, 
        prepare_base_model_config: PrepareBaseModelConfig):
            self.config = prepare_base_model_config
            
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            include_top=self.config.params_include_top,
            weights=self.config.params_weights,
            input_shape=self.config.params_image_size,
        )
        
        self.save_model(
            path=self.config.base_model_path,
            model=self.model)
        
    @staticmethod
    def _prepare_full_model(
        model,
        classes,
        freeze_all,
        freeze_till,
        learning_rate
        ):
            if freeze_all:
                for layer in model.layers:
                    model.trainable = False
            elif (freeze_till is not None) and (freeze_till > 0):
                for layer in model.layers[:-freeze_till]:
                    model.trainable = False
                    
            print(len(model.layers))
            print("hello there")
                    
            flatten_in = tf.keras.layers.Flatten()(model.output)
            prediction = tf.keras.layers.Dense(
                units=classes,
                activation="softmax"
            )
            
            full_model = tf.keras.models.Model(
                inputs=model.inputs,
                outputs=prediction
            )
            
            full_model.compile(
                optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=["accuracy"]
            )
    
    def update_base_model(self):
        # newly added lines for getting basemodel
        # self.model = tf.keras.models.load_model(
        #     self.config.base_model_path
        # )
        
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        
        # self.save_model(path=self.config.updated_base_model_path,
        #                 model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)    
    

In [7]:
config = ConfigurationManager()
prepare_base_model_config = config.get_prepare_base_model()
prepare_base_model = PrepareBaseModel(prepare_base_model_config=prepare_base_model_config)
# prepare_base_model.get_base_model()
prepare_base_model.update_base_model()


[2022-09-21 10:13:47,136: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-09-21 10:13:47,141: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-21 10:13:47,143: INFO: common]: created directory at: artifacts
[2022-09-21 10:13:47,147: INFO: common]: created directory at: artifacts/prepare_base_model


2022-09-21 10:13:47.213875: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-21 10:13:47.213930: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-21 10:13:47.213963: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mobo): /proc/driver/nvidia/version does not exist


[2022-09-21 10:13:47,791: WARNING: hdf5_format]: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
19
hello there


In [14]:
prepare_base_model.full_model

In [ ]:
# my understanding of how prepare base model works

class PrepareBaseModel:
    def __init__(self, prepare_base_model_config: PrepareBaseModelConfig):
        self.config = prepare_base_model_config
        
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            include_top=self.config.params_include_top,
            weights=self.config.params_weights,
            # classes=self.config.params_classes
        )
        self.model.save() # save the model
        
    def update_base_model(
        self, 
        model: tf.keras.models, 
        classes: int, 
        freeze_all: bool,
        freeze_till: int):
            if freeze_all:
                for layer in model.layers:
                    layer.trainable = False
            elif (freeze_till is not None) and (freeze_till > 0):
                for layer in model.layers[:-freeze_till]:
                    layer.trainable = False
            
            flatten_in = tf.keras.layers.Flatten()(model.outputs)
            prediction = tf.keras.layers.Dense(2, activation="categorical_crossentropy")
            tf.keras.models.Model()

2022-09-20 13:04:43.208873: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-20 13:04:43.239136: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-20 13:04:43.239348: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mobo): /proc/driver/nvidia/version does not exist


weights: 2
trainable_weights: 2
non_trainable_weights: 0


In [47]:
class Person:
    
    @staticmethod
    def name(name: str) -> str:
        return f"your name is {name}"
    
    @classmethod
    def surname(cls, lname: str) -> str:
        return f"your surname is {lname}"

In [48]:
p = Person()

In [50]:
Person.surname("sam")

'your surname is sam'

In [51]:
p.surname("paul")

'your surname is paul'

In [54]:
x = [2,3,4,5,2,2]
x[:-2]

[2, 3, 4, 5]